In [1]:
import PIL.Image as Image
import pandas as pd
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision.models import AlexNet_Weights
from torchvision import transforms as T

epochs = 7
batch_size_train = 16
batch_size_test = 100
lr = 0.001
weight_decay = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = T.Compose([
    T.ToTensor(),
    T.Resize([224,224]),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

class MyDataset(Dataset):
    def __init__(self,transform=None,string="train"):
        self.imgs_path = "/content/cats_and_dogs_filtered/" + string + "/"
        file_list = glob.glob(self.imgs_path+"*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path+"/*.jpg"):
                self.data.append([img_path,class_name])
        self.class_map = {"dogs":0,"cats":1}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        img_path,class_name = self.data[idx]
        img = Image.open(img_path)
        class_id = self.class_map[class_name]
        class_id = torch.tensor(class_id)
        if self.transform:
            img = self.transform(img)
        return img,class_id

def get_model():
    model = torch.hub.load('pytorch/vision:v0.10.0','alexnet',weights=AlexNet_Weights.DEFAULT)
    model.features.requires_grad=False
    model.classifier = nn.Sequential(
    *model.classifier[:-1],
    nn.Linear(4096,2,bias=True))
    return model

def train(model,train_loader,criterion,optimizer,device="cpu"):
    model.train()
    running_loss = 0.0
    correct = 0
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*len(data)
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(train_loader.dataset)
    acc = 100*correct/len(train_loader.dataset)
    return acc,running_loss

def test(model,test_loader,criterion,device="cpu"):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device),target.to(device)
            output = model(data)
            running_loss += criterion(output,target).item()*len(data)
            pred = output.data.max(1,keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    return acc,running_loss

train_dataset = MyDataset(transform,"train")
train_loader = DataLoader(train_dataset,batch_size = batch_size_train, shuffle=True)
test_dataset = MyDataset(transform,"validation")
test_loader = DataLoader(test_dataset,batch_size = batch_size_test, shuffle=True)

modelA = get_model()
modelA = modelA.to(device)
print();print();
print(modelA)

criterion = nn.CrossEntropyLoss()
optimizerA = optim.SGD(modelA.parameters(),lr=lr,weight_decay=weight_decay)
for epoch in range(1,epochs+1):
    print("Epoch: ",epoch)
    print("\tTraining: ")
    train_acc, train_loss = train(modelA, train_loader, criterion, optimizerA, device)
    print("\t\tAccuracy: {:.4}%".format(train_acc))
    print("\t\tLoss: {:.4}".format(train_loss))

    print("\tValidation: ")
    test_acc, test_loss = test(modelA, test_loader, criterion, device)
    print("\t\tAccuracy: {:.4}%".format(test_acc))
    print("\t\tLoss: {:.4}".format(test_loss))

['/content/cats_and_dogs_filtered/train/cats', '/content/cats_and_dogs_filtered/train/dogs']
['/content/cats_and_dogs_filtered/validation/cats', '/content/cats_and_dogs_filtered/validation/dogs']


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 136MB/s]




AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)

In [10]:
regularization = 2

def train_with_regularization(model,train_loader,criterion,optimizer,regularization=2,lamda=0.001,device="cpu"):
    model.train()
    running_loss = 0.0
    correct = 0
    for bacth_idx,(inputs,target) in enumerate(train_loader):
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        optimizer.step()
        loss = criterion(output,target)
        running_loss = loss.item()*len(inputs)
        norm = sum(torch.norm(p,regularization) for p in model.parameters())
        loss = loss + lamda*norm
        loss.backward()
        optimizer.step()
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(train_loader)
    acc = 100. * correct / len(train_loader)
    return acc, running_loss

modelB = get_model()
modelB = modelB.to(device)

optimizerB = optim.SGD(modelB.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    print(f"Epoch: {epoch}")

    print("\tTraining: ")
    train_acc, train_loss = train_with_regularization(modelB, train_loader, criterion, optimizerB, regularization, weight_decay, device)
    print("\t\tAccuracy: {:.4}%".format(train_acc))
    print("\t\tLoss: {:.4}".format(train_loss))

    print("\tValidation: ")
    test_acc, test_loss = test(modelB, test_loader, criterion, device)
    print("\t\tAccuracy: {:.4}%".format(test_acc))
    print("\t\tLoss: {:.4}".format(test_loss))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch: 1
	Training: 
		Accuracy: 1.464e+03%
		Loss: 0.01461
	Validation: 
		Accuracy: 96.3%
		Loss: 0.1102
Epoch: 2
	Training: 
		Accuracy: 1.53e+03%
		Loss: 0.01061
	Validation: 
		Accuracy: 96.5%
		Loss: 0.09395
Epoch: 3
	Training: 
		Accuracy: 1.54e+03%
		Loss: 0.006713
	Validation: 
		Accuracy: 96.0%
		Loss: 0.09202
Epoch: 4
	Training: 
		Accuracy: 1.556e+03%
		Loss: 0.00773
	Validation: 
		Accuracy: 96.4%
		Loss: 0.08635
Epoch: 5
	Training: 
		Accuracy: 1.562e+03%
		Loss: 0.001947
	Validation: 
		Accuracy: 96.8%
		Loss: 0.08097
Epoch: 6
	Training: 
		Accuracy: 1.575e+03%
		Loss: 0.02008
	Validation: 
		Accuracy: 95.9%
		Loss: 0.08674
Epoch: 7
	Training: 
		Accuracy: 1.582e+03%
		Loss: 0.004256
	Validation: 
		Accuracy: 96.8%
		Loss: 0.08211


In [11]:
regularization = 1

def train_with_regularization_1(model,train_loader,criterion,optimizer,regularization=2,lamda=0.001,device="cpu"):
    model.train()
    running_loss = 0.0
    correct = 0
    for bacth_idx,(inputs,target) in enumerate(train_loader):
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        optimizer.step()
        loss = criterion(output,target)
        running_loss = loss.item()*len(inputs)
        norm = sum(torch.norm(p,regularization) for p in model.parameters())
        loss = loss + lamda*norm
        loss.backward()
        optimizer.step()
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    running_loss /= len(train_loader)
    acc = 100. * correct / len(train_loader)
    return acc, running_loss

modelC = get_model()
modelC = modelC.to(device)

optimizerC = optim.SGD(modelC.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    print(f"Epoch: {epoch}")

    print("\tTraining: ")
    train_acc, train_loss = train_with_regularization_1(modelC, train_loader, criterion, optimizerC, regularization, weight_decay, device)
    print("\tAccuracy: {:.4}%".format(train_acc))
    print("\tLoss: {:.4}".format(train_loss))

    print("\tValidation: ")
    test_acc, test_loss = test(modelC, test_loader, criterion, device)
    print("\tAccuracy: {:.4}%".format(test_acc))
    print("\tLoss: {:.4}".format(test_loss))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch: 1
	Training: 
		Accuracy: 1.455e+03%
		Loss: 0.00657
	Validation: 
		Accuracy: 96.1%
		Loss: 0.1126
Epoch: 2
	Training: 
		Accuracy: 1.535e+03%
		Loss: 0.009093
	Validation: 
		Accuracy: 96.3%
		Loss: 0.09783
Epoch: 3
	Training: 
		Accuracy: 1.545e+03%
		Loss: 0.01068
	Validation: 
		Accuracy: 96.3%
		Loss: 0.08779
Epoch: 4
	Training: 
		Accuracy: 1.56e+03%
		Loss: 0.007282
	Validation: 
		Accuracy: 96.3%
		Loss: 0.08739
Epoch: 5
	Training: 
		Accuracy: 1.562e+03%
		Loss: 0.01311
	Validation: 
		Accuracy: 96.5%
		Loss: 0.08619
Epoch: 6
	Training: 
		Accuracy: 1.57e+03%
		Loss: 0.01763
	Validation: 
		Accuracy: 96.5%
		Loss: 0.08133
Epoch: 7
	Training: 
		Accuracy: 1.574e+03%
		Loss: 0.001077
	Validation: 
		Accuracy: 96.4%
		Loss: 0.07915


In [12]:
import PIL.Image as Image
import pandas as pd
import glob
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset,DataLoader
from torchvision.models import AlexNet_Weights
from torchvision import transforms as T
import matplotlib.pyplot as plt

epochs = 10
batch_size_train = 16
batch_size_test = 100
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = T.Compose([
    T.ToTensor(),
    T.Resize([224,224]),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
classes = {'dogs':0,'cats':1}

no_dropout = {'train':[],'test':[]}
dropout = {'train':[],'test':[]}

class MyDataset(Dataset):
  def __init__(self,transform=None,string="train"):
    self.imgs_path = "/content/cats_and_dogs_filtered/" + string + "/"
    file_list = glob.glob(self.imgs_path+"*")
    self.data = []
    for class_path in file_list:
      class_name = class_path.split("/")[-1]
      for img_path in glob.glob(class_path+"/*.jpg"):
        self.data.append([img_path,class_name])
    self.class_map = {"dogs":0,"cats":1}
    self.transform=transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    img_path,class_name = self.data[idx]
    img = Image.open(img_path)
    class_id = self.class_map[class_name]
    class_id = torch.tensor(class_id)
    if self.transform:
      img = self.transform(img)
    return img,class_id

def get_model():
  model = torch.hub.load("pytorch/vision:v0.10.0","alexnet",weights=AlexNet_Weights.DEFAULT)
  model.features.requires_grad = False
  model.classifier = nn.Sequential(
      nn.Linear(256*6*6,4096),
      nn.ReLU(inplace=True),
      nn.Linear(4096,4096),
      nn.ReLU(inplace=True),
      nn.Linear(4096,2),
  )
  return model

model1 = get_model().to(device)
print(model1)

dropout = 0.1
def model_with_dropout(dropout=0.5):
  model = torch.hub.load("pytorch/vision:v0.10.0","alexnet",weights=AlexNet_Weights.DEFAULT)
  model.features.requires_grad = False
  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout),
      nn.Linear(256*6*6,4096),
      nn.ReLU(inplace=True),
      nn.Dropout(p=dropout),
      nn.Linear(4096,4096),
      nn.ReLU(inplace=True),
      nn.Linear(4096,2),
  )
  return model

model2 = model_with_dropout().to(device)
print(model2)

def train(model,train_loader,criterion,optimizer,deviec="cpu"):
  model.train()
  running_loss = 0.0
  correct = 0
  for batch_idx, (inputs,target) in enumerate(train_loader):
    inputs,target = inputs.to(device),target.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,target)
    loss.backward()
    optimizer.step()
    preds = outputs.data.max(1,keepdim=True)[1]
    running_loss += loss.item()*len(inputs)
    correct += preds.eq(target.data.view_as(preds)).sum()
  running_loss /= len(train_loader)
  acc = 100. * correct / len(train_loader)
  return acc,running_loss

def test(model,test_loader,criterion,device="cpu"):
  model.eval()
  running_loss = 0.0
  correct = 0
  with torch.no_grad():
    for inputs,target in test_loader:
      inputs,target = inputs.to(device),target.to(device)
      output = model(inputs)
      loss = criterion(output,target)
      running_loss += loss.item()
      pred = output.data.max(1,keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  running_loss /= len(test_loader)*len(inputs)
  acc = 100. * correct / len(test_loader)
  return acc, running_loss

train_dataset = MyDataset(transform,"train")
test_dataset = MyDataset(transform,"validation")
train_loader = DataLoader(train_dataset,batch_size=batch_size_train,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size_test,shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(model1.parameters(),lr=lr)

print("Without Dropout")

for epoch in range(1,epochs+1):
  print(f"Epoch: {epoch}")
  print("Training: ")
  train_acc, train_loss = train(model1, train_loader, criterion, optimizer1, device)
  print("\tAccuracy: {:.4}%".format(train_acc))
  print("\tLoss: {:.4}".format(train_loss))

  print("Validation: ")
  test_acc, test_loss = test(model1, test_loader, criterion, device)
  print("\tAccuracy: {:.4}%".format(test_acc))
  print("\tLoss: {:.4}".format(test_loss))

  no_dropout['train'].append(train_acc.item())
  no_dropout['test'].append(test_acc.item())

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Lin

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

AttributeError: 'Tensor' object has no attribute '4'

In [14]:
print("With Dropout")

optimizer2 = optim.SGD(model2.parameters(),lr)
for epoch in range(1,epochs+1):
  print("Epoch:",epoch)
  print("Training:")
  train_acc, train_loss = train(model2,train_loader,criterion,optimizer2,device)
  print("\tAccuracy: {}%".format(train_acc))
  print("\tLoss: {}%".format(train_loss))
  print("Testing:")
  test_acc, test_loss = test(model2,test_loader,criterion,device)
  print("\tAccuracy: {}%".format(test_acc))
  print("\tLoss: {}%".format(test_loss))

With Dropout
Epoch: 1
Training:
	Accuracy: 1491.80322265625%
	Loss: 2.603708149346172%
Testing:
	Accuracy: 9530.0%
	Loss: 0.0012198191881179809%
Epoch: 2
Training:
	Accuracy: 1498.360595703125%
	Loss: 2.3430086227103333%
Testing:
	Accuracy: 9520.0%
	Loss: 0.001258479818701744%
Epoch: 3
Training:
	Accuracy: 1517.2130126953125%
	Loss: 1.9669015815359403%
Testing:
	Accuracy: 9500.0%
	Loss: 0.0012778423391282558%
Epoch: 4
Training:
	Accuracy: 1530.3277587890625%
	Loss: 1.6987318984064899%
Testing:
	Accuracy: 9560.0%
	Loss: 0.0010391588024795055%
Epoch: 5
Training:
	Accuracy: 1537.704833984375%
	Loss: 1.3447108487159656%
Testing:
	Accuracy: 9560.0%
	Loss: 0.0009538969695568085%
Epoch: 6
Training:
	Accuracy: 1547.5408935546875%
	Loss: 1.2356948857730041%
Testing:
	Accuracy: 9480.0%
	Loss: 0.0011179698035120964%
Epoch: 7
Training:
	Accuracy: 1549.9998779296875%
	Loss: 1.1141316550860152%
Testing:
	Accuracy: 8390.0%
	Loss: 0.005760994732379913%
Epoch: 8
Training:
	Accuracy: 1549.9998779296875%